Check the Domain Expansion hand sign: https://www.wikihow.com/Domain-Expansion-Hand-Sign

In [1]:
#pip install opencv-python

In [2]:
#pip install mediapipe

In [3]:
#pip install tensorflow

In [4]:
#pip install pygame

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import os
import pygame
import requests
import time
import threading
from io import BytesIO
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class JujutsuKaisenGestureRecognition:
    def __init__(self):
        # Initialize MediaPipe Hands
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils
        
        # Domain definitions 
        self.domain_signs = {
            "infinite_void": "Infinite Void (Gojo Satoru)",
            "chimera_shadow": "Chimera Shadow (Megumi Fushiguro)",
            "malevolent_shrine": "Malevolent Shrine (Sukuna)"
        }
        
        # Paths for data storage
        self.data_dir = "jujutsu_data"
        self.model_path = "jujutsu_model.pkl"
        self.sound_dir = "sounds"
        self.image_dir = "images"
        
        # Ensure directories exist
        for directory in [self.data_dir, self.sound_dir, self.image_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory)
        
        # Initialize pygame for sound (if available)
        try:
            pygame.mixer.init()
            self.sounds_available = True
        except:
            print("Sound functionality not available. Continuing without sound effects.")
            self.sounds_available = False
        
        # Load or create model
        self.model = None
        if os.path.exists(self.model_path):
            self.load_model()
        
        # Variables for data collection
        self.collecting_data = False
        self.current_domain = ""
        self.collected_frames = []
        self.recording_countdown = 0
        
        # Variables for prediction
        self.last_prediction = None
        self.prediction_confidence = 0
        self.prediction_time = 0
        self.cooldown_time = 3  # seconds between predictions
        
        # For prediction smoothing
        self.hand_history = []
        self.hand_history_size = 5
        
        print("✅ Jujutsu Kaisen Domain Expansion recognition system initialized")

    def extract_hand_features(self, frame):
        """Extract features from hand landmarks using MediaPipe Hands"""
        # Convert image to RGB for MediaPipe
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the image with MediaPipe Hands
        results = self.hands.process(image_rgb)
        
        # Create a fixed-size feature vector (42 features per hand)
        features = np.zeros(84)
        
        hands_detected = False
        
        if results.multi_hand_landmarks:
            hands_detected = True
            
            # Draw hand landmarks on the frame
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    self.mp_hands.HAND_CONNECTIONS
                )
            
            # Extract features from up to 2 hands
            for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks[:2]):
                for i, landmark in enumerate(hand_landmarks.landmark):
                    # Only store x and y coordinates (ignore z)
                    base_idx = hand_idx * 42  # 21 landmarks * 2 values per landmark
                    features[base_idx + i*2] = landmark.x
                    features[base_idx + i*2 + 1] = landmark.y
        
        return features, frame, hands_detected

    def collect_data(self):
        """Run the data collection interface"""
        cv2.namedWindow("Jujutsu Kaisen Data Collector", cv2.WINDOW_NORMAL)
        cap = cv2.VideoCapture(0)
        
        # Check if camera opened successfully
        if not cap.isOpened():
            print("Error: Could not open camera. Please check your camera connection.")
            return
            
        # Set camera properties for better performance
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        print("\n== DATA COLLECTION MODE ==")
        print("Press keys 1-3 to select a domain to collect data for:")
        print("1: Infinite Void (Gojo)")
        print("2: Chimera Shadow (Megumi)")
        print("3: Malevolent Shrine (Sukuna)")
        print("Press 'R' to start/stop recording frames (1-second delay to prepare)")
        print("Press 'S' to save the collected data")
        print("Press 'Q' to quit")
        
        domains = list(self.domain_signs.keys())
        domain_counts = {domain: 0 for domain in domains}
        
        # Count existing data
        for domain in domains:
            domain_path = os.path.join(self.data_dir, f"{domain}.npy")
            if os.path.exists(domain_path):
                try:
                    existing_data = np.load(domain_path)
                    domain_counts[domain] = len(existing_data)
                    print(f"Found {len(existing_data)} existing samples for {domain}")
                except Exception as e:
                    print(f"Error loading existing data for {domain}: {e}")
        
        countdown_start = 0
        
        while True:
            # Read a frame
            ret, frame = cap.read()
            
            # Check if frame was read successfully
            if not ret:
                print("Error: Failed to capture frame from camera")
                # Try to reopen the camera
                cap.release()
                time.sleep(1)
                cap = cv2.VideoCapture(0)
                if not cap.isOpened():
                    print("Error: Could not reopen camera. Exiting data collection.")
                    break
                continue
            
            # Flip the frame for a selfie-view
            frame = cv2.flip(frame, 1)
            
            try:
                # Extract hand features
                features, processed_frame, hands_detected = self.extract_hand_features(frame)
                
                # Handle recording countdown
                current_time = time.time()
                if self.recording_countdown > 0:
                    # Show countdown
                    remaining = max(0, 1 - (current_time - countdown_start))
                    if remaining > 0:
                        cv2.putText(processed_frame, f"Starting in: {remaining:.1f}s", 
                                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                    else:
                        self.recording_countdown = 0
                        self.collecting_data = True
                        print(f"Started recording frames for {self.current_domain}")
                
                # If we're collecting data, store the features
                if self.collecting_data:
                    if hands_detected:
                        self.collected_frames.append((features, self.current_domain))
                        # Display recording indicator
                        cv2.putText(processed_frame, f"Recording: {self.current_domain} ({len(self.collected_frames)})",
                                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    else:
                        # Warn that hands aren't detected
                        cv2.putText(processed_frame, "No hands detected - not recording", 
                                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                
                # Display instructions and status
                cv2.putText(processed_frame, "1-3: Select Domain, R: Record, S: Save, Q: Quit",
                            (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                
                # Display current data counts and selected domain
                y_pos = 90
                for i, domain in enumerate(domains):
                    count_text = f"{i+1}: {domain} - {domain_counts[domain] + (len(self.collected_frames) if self.current_domain == domain and self.collecting_data else 0)}"
                    color = (0, 255, 0) if domain == self.current_domain else (200, 200, 200)
                    cv2.putText(processed_frame, count_text, (10, y_pos), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
                    y_pos += 30
                
                # Show if hands are detected
                hand_status = "Hands Detected: Yes" if hands_detected else "Hands Detected: No"
                cv2.putText(processed_frame, hand_status, (10, frame.shape[0] - 40), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0) if hands_detected else (0, 0, 255), 1)
                
                # Show the frame
                cv2.imshow("Jujutsu Kaisen Data Collector", processed_frame)
            except Exception as e:
                print(f"Error in processing frame: {e}")
                # Show a simpler frame if there's an error
                cv2.putText(frame, f"Processing error: {str(e)[:50]}...", 
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.imshow("Jujutsu Kaisen Data Collector", frame)
            
            # Process key presses
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('r'):
                if self.collecting_data:
                    self.collecting_data = False
                    print(f"Stopped recording. Collected {len(self.collected_frames)} frames for {self.current_domain}")
                elif self.current_domain:
                    # Start countdown
                    self.recording_countdown = 1
                    countdown_start = current_time
                else:
                    print("Please select a domain first (press 1-3)")
            elif key == ord('s'):
                if self.collected_frames:
                    try:
                        self.save_collected_data()
                        # Update counts
                        domain_counts[self.current_domain] += len(self.collected_frames)
                        self.collected_frames = []
                        self.collecting_data = False
                    except Exception as e:
                        print(f"Error saving data: {e}")
                else:
                    print("No data to save")
            elif key in [ord('1'), ord('2'), ord('3')]:
                domain_idx = int(chr(key)) - 1
                if domain_idx < len(domains):
                    self.current_domain = domains[domain_idx]
                    print(f"Selected domain: {self.current_domain}")
                    if self.collecting_data:
                        self.collecting_data = False
                        print("Recording stopped due to domain change")
        
        # Clean up
        cap.release()
        cv2.destroyAllWindows()

    def save_collected_data(self):
        """Save the collected frames to disk"""
        if not self.collected_frames:
            print("No data to save")
            return
        
        domain_path = os.path.join(self.data_dir, f"{self.current_domain}.npy")
        labels_path = os.path.join(self.data_dir, f"{self.current_domain}_labels.npy")
        
        # Extract features and labels
        features = np.array([f[0] for f in self.collected_frames])
        labels = np.array([f[1] for f in self.collected_frames])
        
        # Check if we already have data for this domain
        if os.path.exists(domain_path) and os.path.exists(labels_path):
            try:
                existing_data = np.load(domain_path)
                existing_labels = np.load(labels_path)
                features = np.vstack((existing_data, features))
                labels = np.hstack((existing_labels, labels))
                print(f"Appending {len(self.collected_frames)} frames to existing data")
            except Exception as e:
                print(f"Error loading existing data: {e}. Creating new data file.")
        else:
            print(f"Saving {len(self.collected_frames)} frames as new data")
        
        # Save the data
        np.save(domain_path, features)
        np.save(labels_path, labels)
        
        print(f"✅ Saved {len(features)} samples for {self.current_domain}")
        self.collected_frames = []

    def train_model(self):
        """Train the gesture recognition model"""
        print("\n== TRAINING MODEL ==")
        
        # Check if we have data
        if not os.path.exists(self.data_dir):
            print("❌ No data directory found")
            return
        
        # Collect all data
        features = []
        labels = []
        
        for domain in self.domain_signs.keys():
            domain_path = os.path.join(self.data_dir, f"{domain}.npy")
            labels_path = os.path.join(self.data_dir, f"{domain}_labels.npy")
            
            if os.path.exists(domain_path) and os.path.exists(labels_path):
                try:
                    domain_features = np.load(domain_path)
                    domain_labels = np.load(labels_path)
                    
                    features.append(domain_features)
                    labels.append(domain_labels)
                    print(f"Loaded {len(domain_features)} samples for {domain}")
                except Exception as e:
                    print(f"Error loading data for {domain}: {e}")
        
        if not features:
            print("❌ No training data found")
            return
        
        # Combine all data
        try:
            X = np.vstack(features)
            y = np.hstack(labels)
            
            print(f"Total dataset: {len(X)} samples")
            
            # Split data into train and test sets
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
            # Train a RandomForest classifier
            print("Training RandomForest classifier...")
            self.model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
            self.model.fit(X_train, y_train)
            
            # Evaluate the model
            y_pred = self.model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Model accuracy: {accuracy:.4f}")
            
            # Save the model
            with open(self.model_path, 'wb') as f:
                pickle.dump(self.model, f)
                
            print(f"✅ Model saved to {self.model_path}")
        except Exception as e:
            print(f"❌ Error training model: {e}")

    def load_model(self):
        """Load a trained model"""
        try:
            with open(self.model_path, 'rb') as f:
                self.model = pickle.load(f)
            print(f"✅ Loaded model from {self.model_path}")
        except Exception as e:
            print(f"❌ Could not load model: {e}")
            self.model = None

    def smooth_predictions(self, prediction, confidence):
        """Smooth predictions over time to reduce jitter"""
        # Add current prediction to history
        self.hand_history.append((prediction, confidence))
        if len(self.hand_history) > self.hand_history_size:
            self.hand_history.pop(0)
            
        # Count occurrences of each prediction
        prediction_counts = {}
        total_confidence = {}
        
        for pred, conf in self.hand_history:
            if pred not in prediction_counts:
                prediction_counts[pred] = 0
                total_confidence[pred] = 0
            prediction_counts[pred] += 1
            total_confidence[pred] += conf
            
        # Find the most common prediction
        max_count = 0
        smoothed_prediction = None
        smoothed_confidence = 0
        
        for pred, count in prediction_counts.items():
            if count > max_count:
                max_count = count
                smoothed_prediction = pred
                smoothed_confidence = total_confidence[pred] / count
                
        # Return the smoothed prediction if it's stable enough
        if max_count >= 0.6 * len(self.hand_history):
            return smoothed_prediction, smoothed_confidence
        else:
            return None, 0

    def start_recognition(self):
        """Start the real-time recognition system"""
        if self.model is None:
            print("❌ No model loaded. Please train a model first.")
            return
        
        print("\n== RECOGNITION MODE ==")
        print("Performing real-time Domain Expansion recognition")
        print("Press 'Q' to quit")
        
        cv2.namedWindow("Jujutsu Kaisen Domain Recognition", cv2.WINDOW_NORMAL)
        cap = cv2.VideoCapture(0)
        
        # Check if camera opened successfully
        if not cap.isOpened():
            print("Error: Could not open camera. Please check your camera connection.")
            return
            
        # Set camera properties for better performance
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        # Initialize hand history for smooth predictions
        self.hand_history = []
        
        showing_effect = False
        effect_start_time = 0
        effect_duration = 5  # seconds
        
        # Prepare domain expansion images
        domain_images = {}
        for domain in self.domain_signs.keys():
            img_path = os.path.join(self.image_dir, f"{domain}.jpg")
            if os.path.exists(img_path):
                try:
                    domain_images[domain] = cv2.imread(img_path)
                    print(f"Loaded domain image for {domain}")
                except Exception as e:
                    print(f"Error loading image for {domain}: {e}")
                    # Create a basic colored background as fallback
                    if domain == "infinite_void":
                        # Blue background for Gojo
                        domain_images[domain] = np.ones((480, 640, 3), dtype=np.uint8) * np.array([128, 0, 0], dtype=np.uint8)
                    elif domain == "chimera_shadow":
                        # Dark background for Megumi
                        domain_images[domain] = np.ones((480, 640, 3), dtype=np.uint8) * np.array([0, 0, 0], dtype=np.uint8)
                    elif domain == "malevolent_shrine":
                        # Red background for Sukuna
                        domain_images[domain] = np.ones((480, 640, 3), dtype=np.uint8) * np.array([0, 0, 128], dtype=np.uint8)
            else:
                print(f"No image found for {domain}, creating basic background")
                # Create a basic colored background
                if domain == "infinite_void":
                    # Blue background for Gojo
                    domain_images[domain] = np.ones((480, 640, 3), dtype=np.uint8) * np.array([128, 0, 0], dtype=np.uint8)
                elif domain == "chimera_shadow":
                    # Dark background for Megumi
                    domain_images[domain] = np.ones((480, 640, 3), dtype=np.uint8) * np.array([0, 0, 0], dtype=np.uint8)
                elif domain == "malevolent_shrine":
                    # Red background for Sukuna
                    domain_images[domain] = np.ones((480, 640, 3), dtype=np.uint8) * np.array([0, 0, 128], dtype=np.uint8)
        
        while True:
            # Read a frame
            ret, frame = cap.read()
            
            # Check if frame was read successfully
            if not ret:
                print("Error: Failed to capture frame from camera")
                # Try to reopen the camera
                cap.release()
                time.sleep(1)
                cap = cv2.VideoCapture(0)
                if not cap.isOpened():
                    print("Error: Could not reopen camera. Exiting recognition.")
                    break
                continue
            
            # Flip the frame for a selfie-view
            frame = cv2.flip(frame, 1)
            
            try:
                # If we're showing an effect, overlay it
                current_time = time.time()
                if showing_effect and current_time - effect_start_time < effect_duration:
                    # Continue showing effect
                    domain = self.last_prediction
                    
                    # Use the domain image if available
                    if domain in domain_images and domain_images[domain] is not None:
                        effect_frame = domain_images[domain].copy()
                        
                        # Calculate opacity based on time (fade in and out)
                        elapsed = current_time - effect_start_time
                        if elapsed < 0.8:  # Fade in
                            alpha = elapsed / 0.8
                        elif elapsed > effect_duration - 0.8:  # Fade out
                            alpha = (effect_duration - elapsed) / 0.8
                        else:
                            alpha = 1.0
                        
                        # Create a combined frame
                        h, w = frame.shape[:2]
                        effect_frame = cv2.resize(effect_frame, (w, h))
                        
                        # Overlay with transparency
                        frame = cv2.addWeighted(frame, 1 - 0.85 * alpha, effect_frame, 0.85 * alpha, 0)
                        
                        # Add announcement text
                        domain_name = self.domain_signs[domain]
                        # Add a dark background for text readability
                        text_bg = np.zeros_like(frame)
                        cv2.rectangle(text_bg, (0, h-70), (w, h), (0, 0, 0), -1)
                        frame = cv2.addWeighted(frame, 1, text_bg, 0.7, 0)
                        
                        # Add domain expansion text
                        cv2.putText(frame, "DOMAIN EXPANSION", 
                                    (int(w/2)-220, h-40), cv2.FONT_HERSHEY_DUPLEX, 
                                    1.5, (0, 0, 255), 3)
                        
                        # Add domain name
                        cv2.putText(frame, domain_name, 
                                    (int(w/2)-200, h-10), cv2.FONT_HERSHEY_SIMPLEX, 
                                    0.8, (0, 255, 255), 2)
                else:
                    showing_effect = False
                    
                    # Extract hand features and make a prediction
                    features, frame, hands_detected = self.extract_hand_features(frame)
                    
                    # Only predict if we have a model and hands are detected
                    if hands_detected and current_time - self.prediction_time > self.cooldown_time:
                        # Make prediction
                        prediction = self.model.predict([features])[0]
                        
                        # Get prediction probabilities
                        probs = self.model.predict_proba([features])[0]
                        confidence = np.max(probs)
                        
                        # Apply smoothing
                        smooth_prediction, smooth_confidence = self.smooth_predictions(prediction, confidence)
                        
                        # Only consider high confidence predictions
                        if smooth_prediction and smooth_confidence > 0.7:
                            print(f"Detected: {smooth_prediction} ({smooth_confidence:.2f})")
                            self.last_prediction = smooth_prediction
                            self.prediction_confidence = smooth_confidence
                            self.prediction_time = current_time
                            
                            # Trigger effect and sound
                            showing_effect = True
                            effect_start_time = current_time
                            
                            # Play sound if available
                            if self.sounds_available:
                                try:
                                    if "announcement" in self.sounds:
                                        self.sounds["announcement"].play()
                                except Exception as e:
                                    print(f"Error playing sound: {e}")
                    
                    # Show hand detection status and confidence
                    if hands_detected:
                        cv2.putText(frame, "Hands Detected", (10, 30), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                        
                        # If we've made a recent prediction, show it
                        if current_time - self.prediction_time < 10:
                            confidence_text = f"Last: {self.last_prediction} ({self.prediction_confidence:.2f})"
                            cv2.putText(frame, confidence_text, (10, 60), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                    else:
                        cv2.putText(frame, "No Hands Detected", (10, 30), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                
                # Display instructions
                cv2.putText(frame, "Press 'Q' to quit", (10, frame.shape[0] - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                
                # Show the frame
                cv2.imshow("Jujutsu Kaisen Domain Recognition", frame)
            except Exception as e:
                print(f"Error processing frame: {e}")
                # Create a basic error frame
                error_frame = np.ones((480, 640, 3), dtype=np.uint8) * 127  # Gray background
                cv2.putText(error_frame, f"Error: {str(e)[:50]}...", (50, 240), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(error_frame, "Press 'Q' to quit", (50, 270), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.imshow("Jujutsu Kaisen Domain Recognition", error_frame)
            
            # Process key presses
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
        
        # Clean up
        cap.release()
        cv2.destroyAllWindows()

    def run(self):
        """Main menu interface"""
        while True:
            print("\n== JUJUTSU KAISEN DOMAIN EXPANSION RECOGNITION ==")
            print("1: Collect Training Data")
            print("2: Train Model")
            print("3: Start Recognition")
            print("4: Exit")
            
            try:
                choice = input("Select an option: ")
                
                if choice == '1':
                    self.collect_data()
                elif choice == '2':
                    self.train_model()
                elif choice == '3':
                    self.start_recognition()
                elif choice == '4':
                    print("Exiting...")
                    break
                else:
                    print("Invalid choice, please try again")
            except Exception as e:
                print(f"Error: {e}")
                print("Continuing to main menu...")

if __name__ == "__main__":
    try:
        jjk = JujutsuKaisenGestureRecognition()
        jjk.run()
    except Exception as e:
        print(f"Fatal error: {e}")

pygame 2.6.1 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
✅ Loaded model from jujutsu_model.pkl
✅ Jujutsu Kaisen Domain Expansion recognition system initialized

== JUJUTSU KAISEN DOMAIN EXPANSION RECOGNITION ==
1: Collect Training Data
2: Train Model
3: Start Recognition
4: Exit


Select an option:  1



== DATA COLLECTION MODE ==
Press keys 1-3 to select a domain to collect data for:
1: Infinite Void (Gojo)
2: Chimera Shadow (Megumi)
3: Malevolent Shrine (Sukuna)
Press 'R' to start/stop recording frames (1-second delay to prepare)
Press 'S' to save the collected data
Press 'Q' to quit
Found 3518 existing samples for infinite_void
Selected domain: chimera_shadow
Started recording frames for chimera_shadow
Saving 3538 frames as new data
✅ Saved 3538 samples for chimera_shadow
Selected domain: malevolent_shrine
Started recording frames for malevolent_shrine
Saving 4031 frames as new data
✅ Saved 4031 samples for malevolent_shrine

== JUJUTSU KAISEN DOMAIN EXPANSION RECOGNITION ==
1: Collect Training Data
2: Train Model
3: Start Recognition
4: Exit


Select an option:  2



== TRAINING MODEL ==
Loaded 3518 samples for infinite_void
Loaded 3538 samples for chimera_shadow
Loaded 4031 samples for malevolent_shrine
Total dataset: 11087 samples
Training RandomForest classifier...
Model accuracy: 0.9995
✅ Model saved to jujutsu_model.pkl

== JUJUTSU KAISEN DOMAIN EXPANSION RECOGNITION ==
1: Collect Training Data
2: Train Model
3: Start Recognition
4: Exit


Select an option:  3



== RECOGNITION MODE ==
Performing real-time Domain Expansion recognition
Press 'Q' to quit
Loaded domain image for infinite_void
Loaded domain image for chimera_shadow
Loaded domain image for malevolent_shrine
Detected: infinite_void (0.88)
Error playing sound: 'JujutsuKaisenGestureRecognition' object has no attribute 'sounds'
Detected: malevolent_shrine (0.97)
Error playing sound: 'JujutsuKaisenGestureRecognition' object has no attribute 'sounds'
Detected: malevolent_shrine (0.82)
Error playing sound: 'JujutsuKaisenGestureRecognition' object has no attribute 'sounds'
Detected: malevolent_shrine (0.82)
Error playing sound: 'JujutsuKaisenGestureRecognition' object has no attribute 'sounds'
Detected: malevolent_shrine (0.76)
Error playing sound: 'JujutsuKaisenGestureRecognition' object has no attribute 'sounds'
Detected: infinite_void (0.77)
Error playing sound: 'JujutsuKaisenGestureRecognition' object has no attribute 'sounds'
Detected: infinite_void (0.77)
Error playing sound: 'Jujuts

Select an option:  4


Exiting...
